# Lab 5 : Exploratory Data Analysis

## Use Huggingface to download the dataset 🤗.

In [1]:
!hf download celesca/election_results_2569 --repo-type dataset --local-dir ./

Fetching 3 files: 100%|███████████████████████████| 3/3 [00:02<00:00,  1.27it/s]
Download complete: 100%|██████████████████████| 818k/818k [00:02<00:00, 347kB/s]/Users/beam/Workspace/Course/my-cpe-lab/Y2/CPE232/sessions/05_eda
Download complete: 100%|██████████████████████| 818k/818k [00:02<00:00, 343kB/s]


In [2]:
!unzip -q ./data.zip

checkdir error:  cannot create /Users/beam/Workspace/Course/my-cpe-lab/Y2/CPE232/sessions/05_eda/all_constituency/�+����--ʦ�+
                 Illegal byte sequence
                 unable to process all_constituency/�+����--ʦ�+/.
checkdir error:  cannot create /Users/beam/Workspace/Course/my-cpe-lab/Y2/CPE232/sessions/05_eda/all_constituency/�+����--ʦ�+
                 Illegal byte sequence
                 unable to process all_constituency/�+����--ʦ�+/�+����--ʦ�+ ��� 1/.
checkdir error:  cannot create /Users/beam/Workspace/Course/my-cpe-lab/Y2/CPE232/sessions/05_eda/all_constituency/�+����--ʦ�+
                 Illegal byte sequence
                 unable to process all_constituency/�+����--ʦ�+/�+����--ʦ�+ ��� 1/constituency_candidates.csv.
checkdir error:  cannot create /Users/beam/Workspace/Course/my-cpe-lab/Y2/CPE232/sessions/05_eda/all_constituency/�+����--ʦ�+
                 Illegal byte sequence
                 unable to process all_constituency/�+����--ʦ�+/�+����--ʦ�+ ���

In [3]:
import shutil
from pathlib import Path

# Create data directory
data_dir = Path("data")
data_dir.mkdir(exist_ok=True)

# Extract only CSV files to data directory
data_folders = ["all_constituency", "area", "party", "partylist", "referendum", "seat", "winner"]

for folder in data_folders:
    folder_path = Path(folder)
    if folder_path.exists():
        for csv_file in folder_path.glob("*.csv"):
            shutil.copy(csv_file, data_dir / csv_file.name)
            print(f"Copied {csv_file.name} to data/")


Copied areas.csv to data/
Copied partyStandings.csv to data/
Copied party_list_top2.csv to data/
Copied map.csv to data/
Copied seat500.csv to data/
Copied winner.csv to data/


In [4]:
# Clean up: Remove the original data folders after extraction
for folder in data_folders:
    folder_path = Path(folder)
    if folder_path.exists():
        shutil.rmtree(folder_path)
        print(f"Removed {folder} directory")

# Also remove the zip file if no longer needed
if Path("data.zip").exists():
    Path("data.zip").unlink()
    print("Removed data.zip")

print("\nCSV files are now in the 'data/' directory")


Removed all_constituency directory
Removed area directory
Removed party directory
Removed partylist directory
Removed referendum directory
Removed seat directory
Removed winner directory
Removed data.zip

CSV files are now in the 'data/' directory


## Thai Font in Matplotlib

Download Thai font from the link below and place it in your working directory:
https://drive.google.com/drive/folders/1s_xqY9Wyx9QNNPZkZlTbFlNM5-zQc4iQ

In [ ]:
# Set the path to your Thai font file
# Download from: https://drive.google.com/drive/folders/1s_xqY9Wyx9QNNPZkZlTbFlNM5-zQc4iQ
font_path = "THSarabunNew.ttf"  # Place the font file in your working directory

In [ ]:
# Test Thai Font
import matplotlib.pyplot as plt
from matplotlib import font_manager, rcParams

font_path = "THSarabunNew.ttf"
font_manager.fontManager.addfont(font_path)
rcParams["font.family"] = "TH Sarabun New"

# Context

## 2026 Thai General Election — Factual Background

The 2026 Thai general election was held on **8 February 2026** to choose members of Thailand’s **House of Representatives**.

The election was triggered after the **House of Representatives was dissolved** on **12 December 2025** following political agreements and constitutional conditions.

All **500 seats** in the House were up for election.

The electoral system used two methods:
- **400 seats** elected from constituencies.
- **100 seats** allocated by party-list proportional representation.

Voters cast separate ballots for a **constituency candidate** and a **political party**.

The election was held alongside a **referendum** to decide whether to begin drafting a **new constitution** to replace the current 2017 charter.

The general election and referendum occurred within the legal timeframe after dissolution, with the election date being the **latest possible date allowed** by constitutional rules.

The campaign period included official candidate registration and advance voting before election day.

The House must have a **majority of 251 seats** to form a government.

# Data Preparation
- Data Cleaning
- Data Transformation
- Data Preprocessing
- Data Mapping

## 1. Import Libraries and Load Data

In [19]:
import json
import warnings
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

warnings.filterwarnings("ignore")

# Load all data files
areas = pd.read_csv("data/areas.csv", encoding="utf-8-sig")
winner = pd.read_csv("data/winner.csv")
party_standings = pd.read_csv("data/partyStandings.csv")
seat500 = pd.read_csv("data/seat500.csv")
party_list_top2 = pd.read_csv("data/party_list_top2.csv")
map_data = pd.read_csv("data/map.csv", encoding="utf-8-sig")

print("Data loaded successfully!")
print(f"\nDatasets shapes:")
print(f"  areas: {areas.shape}")
print(f"  winner: {winner.shape}")
print(f"  party_standings: {party_standings.shape}")
print(f"  seat500: {seat500.shape}")
print(f"  party_list_top2: {party_list_top2.shape}")
print(f"  map_data: {map_data.shape}")

Data loaded successfully!

Datasets shapes:
  areas: (7461, 8)
  winner: (400, 8)
  party_standings: (60, 8)
  seat500: (500, 7)
  party_list_top2: (400, 7)
  map_data: (400, 7)


## 2. Data Cleaning

In [20]:
# 2.1 Check for missing values in all datasets
print("=== Missing Values Analysis ===")
datasets = {
    "areas": areas,
    "winner": winner,
    "party_standings": party_standings,
    "seat500": seat500,
    "party_list_top2": party_list_top2,
    "map_data": map_data,
}

for name, df in datasets.items():
    missing = df.isnull().sum()
    missing_pct = (missing / len(df) * 100).round(2)
    if missing.sum() > 0:
        print(f"\n{name}:")
        print(pd.DataFrame({"Missing Count": missing, "Missing %": missing_pct})[missing > 0])
    else:
        print(f"\n{name}: No missing values")

=== Missing Values Analysis ===

areas: No missing values

winner: No missing values

party_standings: No missing values

seat500: No missing values

party_list_top2: No missing values

map_data: No missing values


In [24]:
# 2.2 Check for duplicates
print("Duplicate Analysis")

for name, df in datasets.items():
    duplicates = df.duplicated().sum()
    print(f"{name}: {duplicates} duplicate rows")

# Seat500 duplicate investigation
print("\nSeat500 duplicate")
print(f"Original shape: {seat500.shape}")
print(f"Unique rows: {seat500.drop_duplicates().shape}")
print(f"Duplicates to remove: {len(seat500) - len(seat500.drop_duplicates())}")

# Remove duplicates from seat500
seat500_clean = seat500.drop_duplicates().reset_index(drop=True)
print(f"\nAfter deduplication: {seat500_clean.shape}")

Duplicate Analysis
areas: 0 duplicate rows
winner: 0 duplicate rows
party_standings: 0 duplicate rows
seat500: 0 duplicate rows
party_list_top2: 0 duplicate rows
map_data: 0 duplicate rows

Seat500 duplicate
Original shape: (500, 7)
Unique rows: (500, 7)
Duplicates to remove: 0

After deduplication: (500, 7)


In [25]:
# 2.3 Check data consistency - party codes across datasets
print("Party Code Consistency")

# Extract party codes from different sources
party_codes_standings = set(party_standings["id"].astype(str))
party_codes_seat500 = set(seat500_clean["partyCode"].astype(str))

print(f"Party codes in party_standings: {len(party_codes_standings)}")
print(f"Party codes in seat500: {len(party_codes_seat500)}")

# Check if they match
common_codes = party_codes_standings.intersection(party_codes_seat500)
print(f"Common party codes: {len(common_codes)}")

if party_codes_standings != party_codes_seat500:
    print(f"\nOnly in party_standings: {party_codes_standings - party_codes_seat500}")
    print(f"Only in seat500: {party_codes_seat500 - party_codes_standings}")
else:
    print("\nAll party codes match across datasets!")

Party Code Consistency
Party codes in party_standings: 60
Party codes in seat500: 22
Common party codes: 22

Only in party_standings: {'49', '36', '52', '39', '51', '20', '40', '25', '50', '38', '59', '41', '14', '58', '35', '7', '57', '53', '45', '24', '47', '18', '56', '28', '16', '19', '15', '26', '30', '55', '17', '23', '34', '32', '60', '54', '22', '31'}
Only in seat500: set()


## 3. Data Transformation

In [26]:
# 3.1 Parse JSON columns in winner dataset
def parse_candidates_json(candidates_str):
    """Parse candidates JSON string to extract structured data"""
    try:
        candidates = json.loads(candidates_str.replace("'", '"'))
        winner_data = candidates[0] if candidates else None
        if winner_data:
            return pd.Series(
                {
                    "winner_name": winner_data.get("fullName", ""),
                    "winner_votes": winner_data.get("totalVotes", 0),
                    "winner_percent": winner_data.get("percent", 0),
                    "winner_party_code": winner_data.get("party", {}).get("code", ""),
                    "winner_party_name": winner_data.get("party", {}).get("name", ""),
                    "winner_color": winner_data.get("party", {}).get("color", "#000000"),
                }
            )
    except Exception:
        pass
    return pd.Series(
        {
            "winner_name": None,
            "winner_votes": 0,
            "winner_percent": 0,
            "winner_party_code": None,
            "winner_party_name": None,
            "winner_color": "#000000",
        }
    )


# Apply parsing to winner dataset
winner_parsed = winner["candidates"].apply(parse_candidates_json)
winner_transformed = pd.concat([winner, winner_parsed], axis=1)

print("Winner data parsed successfully!")
print("\nSample parsed data:")
winner_transformed[
    ["name", "winner_name", "winner_party_name", "winner_votes", "winner_percent"]
].head()

Winner data parsed successfully!

Sample parsed data:


,name,winner_name,winner_party_name,winner_votes,winner_percent
0,กรุงเทพมหานคร เขต 1,ปารเมศ วิทยารักษ์สรรค์,พรรคประชาชน,32564,44.39
1,กรุงเทพมหานคร เขต 2,เสกสิทธิ์ แย้มสงวนศักดิ์,พรรคประชาชน,37055,47.08
2,กรุงเทพมหานคร เขต 3,จรยุทธ จตุรพรประสิทธิ์,พรรคประชาชน,32753,45.07
3,กรุงเทพมหานคร เขต 4,ภัณฑิล น่วมเจิม,พรรคประชาชน,30477,43.33
4,กรุงเทพมหานคร เขต 5,ปิติกรณ์ บรรณเภสัช,พรรคประชาชน,43394,50.07


In [ ]:
# 3.2 Parse party_list_top2 JSON column
def parse_top2_parties(parties_str):
    """Parse top 2 parties JSON string"""
    try:
        parties = json.loads(parties_str.replace("'", '"').replace("None", "null"))
        if len(parties) >= 2:
            return pd.Series(
                {
                    "first_party_code": parties[0].get("code", ""),
                    "first_party_name": parties[0].get("name", ""),
                    "first_votes": parties[0].get("totalVotes", 0),
                    "first_percent": parties[0].get("percent", 0),
                    "second_party_code": parties[1].get("code", ""),
                    "second_party_name": parties[1].get("name", ""),
                    "second_votes": parties[1].get("totalVotes", 0),
                    "second_percent": parties[1].get("percent", 0),
                    "vote_margin": parties[0].get("percent", 0) - parties[1].get("percent", 0),
                }
            )
    except Exception:
        pass
    return pd.Series(
        {
            "first_party_code": None,
            "first_party_name": None,
            "first_votes": 0,
            "first_percent": 0,
            "second_party_code": None,
            "second_party_name": None,
            "second_votes": 0,
            "second_percent": 0,
            "vote_margin": 0,
        }
    )


# Apply parsing
top2_parsed = party_list_top2["parties"].apply(parse_top2_parties)
party_list_transformed = pd.concat([party_list_top2, top2_parsed], axis=1)

print("Party list top 2 parsed successfully!")
print("\nSample parsed data:")
party_list_transformed[
    ["name", "first_party_name", "first_percent", "second_party_name", "vote_margin"]
].head()

In [27]:
# 3.3 Parse map_data referendum JSON
def parse_referendum(ref_str):
    """Parse referendum JSON string"""
    try:
        refs = json.loads(ref_str.replace("'", '"'))
        total_votes = sum(r.get("totalVote", 0) for r in refs)
        result = {"total_referendum_votes": total_votes}
        for ref in refs:
            title = ref.get("title", "").replace(" ", "_").lower()
            result[f"ref_{title}_votes"] = ref.get("totalVote", 0)
            result[f"ref_{title}_percent"] = (
                (ref.get("totalVote", 0) / total_votes * 100) if total_votes > 0 else 0
            )
        return pd.Series(result)
    except Exception:
        return pd.Series(
            {
                "total_referendum_votes": 0,
                "ref_เห็นชอบ_votes": 0,
                "ref_ไม่เห็นชอบ_votes": 0,
                "ref_ไม่แสดงความคิดเห็น_votes": 0,
            }
        )


# Apply parsing
referendum_parsed = map_data["referendums"].apply(parse_referendum)
map_transformed = pd.concat([map_data, referendum_parsed], axis=1)

print("Referendum data parsed successfully!")
print("\nSample parsed data:")
map_transformed[["name", "total_referendum_votes", "ref_เห็นชอบ_votes", "ref_ไม่เห็นชอบ_votes"]].head()

Referendum data parsed successfully!

Sample parsed data:


,name,total_referendum_votes,ref_เห็นชอบ_votes,ref_ไม่เห็นชอบ_votes
0,กรุงเทพมหานคร เขต 1,78266.0,47947.0,26432.0
1,กรุงเทพมหานคร เขต 2,80964.0,50937.0,26114.0
2,กรุงเทพมหานคร เขต 3,87252.0,57511.0,25777.0
3,กรุงเทพมหานคร เขต 4,74620.0,46852.0,24062.0
4,กรุงเทพมหานคร เขต 5,105029.0,75322.0,25811.0


In [29]:
# 3.4 Convert data types
print("Converting Data Types")

# Party standings - ensure numeric columns
numeric_cols_standings = [
    "constituencySeats",
    "partyListSeats",
    "totalSeats",
    "totalPartyListVotes",
    "totalConstituencyVotes",
]
for col in numeric_cols_standings:
    party_standings[col] = pd.to_numeric(party_standings[col], errors="coerce")

# Winner transformed
winner_transformed["winner_votes"] = pd.to_numeric(
    winner_transformed["winner_votes"], errors="coerce"
)
winner_transformed["winner_percent"] = pd.to_numeric(
    winner_transformed["winner_percent"], errors="coerce"
)

# Party list transformed
numeric_cols_top2 = [
    "first_votes",
    "first_percent",
    "second_votes",
    "second_percent",
    "vote_margin",
]
for col in numeric_cols_top2:
    party_list_transformed[col] = pd.to_numeric(party_list_transformed[col], errors="coerce")

print("Data types converted successfully!")
print("\nParty standings dtypes:")
print(party_standings[numeric_cols_standings].dtypes)

Converting Data Types
Data types converted successfully!

Party standings dtypes:
constituencySeats         int64
partyListSeats            int64
totalSeats                int64
totalPartyListVotes       int64
totalConstituencyVotes    int64
dtype: object


## 4. Data Preprocessing

In [30]:
# 4.1 Calculate derived metrics for party performance
print("Calculating Derived Metrics")

# Calculate vote share and seat efficiency
party_standings["total_votes"] = (
    party_standings["totalConstituencyVotes"] + party_standings["totalPartyListVotes"]
)
total_all_votes = party_standings["total_votes"].sum()
party_standings["vote_share_percent"] = (
    party_standings["total_votes"] / total_all_votes * 100
).round(2)
party_standings["seat_share_percent"] = (
    party_standings["totalSeats"] / party_standings["totalSeats"].sum() * 100
).round(2)

# Seats per vote ratio (efficiency metric)
party_standings["seats_per_million_votes"] = (
    party_standings["totalSeats"] / (party_standings["total_votes"] / 1000000)
).round(2)

# Constituency vs Party list efficiency
party_standings["constituency_efficiency"] = (
    party_standings["constituencySeats"] / (party_standings["totalConstituencyVotes"] / 1000000)
).round(2)

party_standings["party_list_efficiency"] = (
    party_standings["partyListSeats"] / (party_standings["totalPartyListVotes"] / 1000000 + 0.001)
).round(2)

print("Derived metrics calculated!")
print("\nTop 5 parties by efficiency:")
party_standings[["partyName", "totalSeats", "vote_share_percent", "seats_per_million_votes"]].head()

Calculating Derived Metrics
Derived metrics calculated!

Top 5 parties by efficiency:


,partyName,totalSeats,vote_share_percent,seats_per_million_votes
0,พรรคภูมิใจไทย,193,24.96,12.12
1,พรรคประชาชน,118,27.62,6.70
2,พรรคเพื่อไทย,74,17.12,6.77
3,พรรคกล้าธรรม,58,6.97,13.04
4,พรรคประชาธิปัตย์,22,8.87,3.89


In [31]:
# 4.2 Create constituency-level competitive index
print("Creating Competitive Index")

# Merge winner and party_list_top2 to get margin data
constituency_analysis = winner_transformed.merge(
    party_list_transformed[
        ["id", "first_party_code", "vote_margin", "first_percent", "second_percent"]
    ],
    left_on="id",
    right_on="id",
    how="left",
)


# Define competitiveness categories
def categorize_competitiveness(margin):
    if pd.isna(margin):
        return "Unknown"
    elif margin < 5:
        return "Highly Competitive"
    elif margin < 15:
        return "Competitive"
    elif margin < 30:
        return "Moderate"
    else:
        return "Safe"


constituency_analysis["competitiveness"] = constituency_analysis["vote_margin"].apply(
    categorize_competitiveness
)

# Competitiveness distribution
comp_dist = constituency_analysis["competitiveness"].value_counts()
comp_dist_pct = (comp_dist / len(constituency_analysis) * 100).round(2)

print("Competitiveness distribution:")
print(pd.DataFrame({"Count": comp_dist, "Percentage": comp_dist_pct}))

Creating Competitive Index
Competitiveness distribution:
                    Count  Percentage
competitiveness                      
Competitive           142       35.50
Moderate              120       30.00
Highly Competitive     77       19.25
Safe                   61       15.25


In [32]:
# 4.3 Aggregate by province
print("Province-Level Aggregation")

# Province winners summary
province_summary = (
    constituency_analysis.groupby("provinceName")
    .agg(
        {
            "winner_party_name": lambda x: x.value_counts().index[0],
            "winner_votes": "sum",
            "vote_margin": "mean",
            "id": "count",
        }
    )
    .rename(columns={"id": "num_constituencies"})
)

# Add referendum data if available
if "ref_เห็นชอบ_percent" in map_transformed.columns:
    referendum_by_province = map_transformed.groupby("provinceName").agg(
        {
            "ref_เห็นชอบ_percent": "mean",
            "ref_ไม่เห็นชอบ_percent": "mean",
            "total_referendum_votes": "sum",
        }
    )
    province_summary = province_summary.merge(
        referendum_by_province, left_index=True, right_index=True, how="left"
    )

print("Province summary created!")
print("\nTop 5 provinces by constituencies:")
province_summary.sort_values("num_constituencies", ascending=False).head()

Province-Level Aggregation
Province summary created!

Top 5 provinces by constituencies:


,winner_party_name,winner_votes,vote_margin,num_constituencies,ref_เห็นชอบ_percent,ref_ไม่เห็นชอบ_percent,total_referendum_votes
provinceName,,,,,,,
กรุงเทพมหานคร,พรรคประชาชน,1262690,28.787463,33,66.897205,28.063037,3028202.0
นครราชสีมา,พรรคเพื่อไทย,575931,10.311770,16,59.355476,30.791886,1311893.0
ขอนแก่น,พรรคกล้าธรรม,405518,11.051648,11,66.031277,26.385064,890721.0
อุบลราชธานี,พรรคไทรวมพลัง,447411,7.637505,11,56.147484,34.254308,870591.0
อุดรธานี,พรรคเพื่อไทย,291001,9.370028,10,66.777700,25.126286,679965.0


## 5. Data Mapping

In [33]:
# 5.1 Create master constituency table
print("Creating Master Constituency Table")

# Start with winner data (has constituency info)
master_table = constituency_analysis.copy()

# Add referendum data
referendum_cols = ["id", "total_referendum_votes", "ref_เห็นชอบ_votes", "ref_ไม่เห็นชอบ_votes"]
if all(col in map_transformed.columns for col in referendum_cols):
    master_table = master_table.merge(map_transformed[referendum_cols], on="id", how="left")

# Add party standings info for the winning party
party_info = party_standings[["id", "partyName", "totalSeats", "vote_share_percent"]].rename(
    columns={"id": "winner_party_code", "partyName": "winner_party_name_full"}
)

# Ensure winner_party_code is string type for matching
master_table["winner_party_code"] = master_table["winner_party_code"].astype(str)
party_info["winner_party_code"] = party_info["winner_party_code"].astype(str)

master_table = master_table.merge(party_info, on="winner_party_code", how="left")

print(f"Master table created with {len(master_table)} rows")
print(f"Columns: {list(master_table.columns)}")

Creating Master Constituency Table
Master table created with 400 rows
Columns: ['id', 'name', 'provinceId', 'partyWinner', 'number', 'provinceName', 'candidates', 'provinceId_key', 'winner_name', 'winner_votes', 'winner_percent', 'winner_party_code', 'winner_party_name', 'winner_color', 'first_party_code', 'vote_margin', 'first_percent', 'second_percent', 'competitiveness', 'total_referendum_votes', 'ref_เห็นชอบ_votes', 'ref_ไม่เห็นชอบ_votes', 'winner_party_name_full', 'totalSeats', 'vote_share_percent']


In [34]:
# 5.2 Create party performance summary by province
print("Party Performance by Province")

party_province = (
    master_table.groupby(["provinceName", "winner_party_name"]).size().reset_index(name="seats_won")
)

party_province_pivot = (
    party_province.pivot(index="provinceName", columns="winner_party_name", values="seats_won")
    .fillna(0)
    .astype(int)
)

# Add total column
party_province_pivot["Total"] = party_province_pivot.sum(axis=1)

# Sort by total
party_province_pivot = party_province_pivot.sort_values("Total", ascending=False)

print("Top 10 provinces by party:")
party_province_pivot.head(10)

Party Performance by Province
Top 10 provinces by party:


winner_party_name,พรรคกล้าธรรม,พรรคประชาชน,พรรคประชาชาติ,พรรคประชาธิปัตย์,พรรคพลังประชารัฐ,พรรคภูมิใจไทย,พรรคเพื่อไทย,พรรคโอกาสใหม่,พรรคไทยสร้างไทย,พรรคไทรวมพลัง,Total
provinceName,,,,,,,,,,,
กรุงเทพมหานคร,0,33,0,0,0,0,0,0,0,0,33
นครราชสีมา,0,3,0,0,0,3,10,0,0,0,16
ขอนแก่น,3,2,0,0,0,3,3,0,0,0,11
อุบลราชธานี,0,0,0,0,0,4,3,0,0,4,11
อุดรธานี,0,1,0,0,0,4,5,0,0,0,10
ชลบุรี,0,5,0,0,0,5,0,0,0,0,10
เชียงใหม่,4,6,0,0,0,0,0,0,0,0,10
บุรีรัมย์,0,0,0,0,0,10,0,0,0,0,10
นครศรีธรรมราช,1,0,0,4,0,4,0,0,0,0,9


In [35]:
# 5.3 Summary statistics of cleaned and processed data
print("Data Preparation Summary")
print("\nFinal datasets ready for analysis:")
print(f"  - master_table: {master_table.shape}")
print(f"  - party_standings: {party_standings.shape}")
print(f"  - province_summary: {province_summary.shape}")
print(f"  - party_province_pivot: {party_province_pivot.shape}")

print("\nKey metrics available:")
print(f"  - Total constituencies: {len(master_table)}")
print(f"  - Total parties: {len(party_standings)}")
print(f"  - Total provinces: {len(province_summary)}")
print(f"  - Competitiveness categories: {constituency_analysis['competitiveness'].nunique()}")

# Check for any remaining issues
print("\nData quality checks:")
print(f"  - Missing winner names: {master_table['winner_name'].isna().sum()}")
print(f"  - Missing party codes: {master_table['winner_party_code'].isna().sum()}")
print(f"  - Zero vote margins: {(master_table['vote_margin'] == 0).sum()}")

Data Preparation Summary

Final datasets ready for analysis:
  - master_table: (400, 25)
  - party_standings: (60, 14)
  - province_summary: (77, 7)
  - party_province_pivot: (77, 11)

Key metrics available:
  - Total constituencies: 400
  - Total parties: 60
  - Total provinces: 77
  - Competitiveness categories: 4

Data quality checks:
  - Missing winner names: 0
  - Missing party codes: 0
  - Zero vote margins: 0
